In [ ]:
from tkinter import *
from PIL import Image
from PIL import ImageDraw
import cv2
import numpy
import matplotlib.pyplot as plt
import joblib

# To update position of the cursure
def locate_xy(event):
    global current_x, current_y
    current_x, current_y = event.x, event.y

# To draw a line
def addLine(event):
    global current_x, current_y
    global draw
    # Draw in the canva
    canvas.create_rectangle((current_x, current_y, event.x, event.y), width=5, fill = color)
    current_x, current_y = event.x, event.y
    # Draw in the image
    draw.rectangle((current_x,current_y,event.x,event.y), fill = color, width = 4, outline="#000001")
    
def new_canvas():
    global draw, image1
    canvas.delete('all')
    image1 = Image.new("RGB", (128, 128), (255, 255, 255))
    draw = ImageDraw.Draw(image1)

# Function to process the image drawn and give resutl of recognizing
def RecognizeDigit ():
    global image1, showTxt, model
    
    # Save the drawn number 
    filename = "my_drawing.jpg"
    image1.save(filename)
    
    # Read the drawn number     
    img = cv2.imread("./"+filename, cv2.IMREAD_GRAYSCALE)
    img = cv2.bitwise_not(img)

    # step 1: Apply Gaussian blur filter
    img = cv2.GaussianBlur(img, (7, 7), 0)

    # step 2 and 3: Extract the Region of interest in the image and center in square
    points = cv2.findNonZero(img)
    x, y, w, h = cv2.boundingRect(points)
    if( w > 0 and h > 0):
        if w > h:
            y = y - (w-h)//2
            img = img[y:y+w, x:x+w]
        else:
            x = x - ( h - w ) // 2
            img = img[y:y+h, x:x+h]

    # step 3: Resize and resample to be 28 x 28 pixels
    img = cv2.resize(img, (28, 28), interpolation = cv2.INTER_CUBIC)
    plt.imshow(img)
    
    img = cv2.resize(img, (28, 28), interpolation = cv2.INTER_CUBIC)
    img = ((numpy.array(img)).reshape(1, 784))
    prediction = model.predict(img)
    
    showTxt.delete(0, "end")
    showTxt.insert(0, prediction[0])
    
    
# Variables to store the current position of the cursure
current_x, current_y = 0,0
# Color of tracing
color = 'black'
    
# Create an empty image and draw object to draw on
# Image will bein memory only, not visible
# Image used to save as jpg image and load it to be used in the model
image1 = Image.new("RGB", (128, 128), (255, 255, 255))
draw = ImageDraw.Draw(image1)

# Load the model
model = joblib.load("Completed_model.joblib")

# Creation of the window tkinter
window = Tk()
window.title('Paint')

# Creation of the entry where the result will be appeared
showTxt = Entry(window)
showTxt.grid(row=0, column= 0, columnspan=2)

# Creation of the canva where the user will draw his number
canvas= Canvas(window,background='white', height= 128, width= 128) 
canvas.grid(row=1,column=0, columnspan=2)

# Creation of the button which is used to earse the canva
buttonDelete = Button(window, text = "Earse", command=new_canvas)
# Creation of the button which is used to recognize the drawn number
buttonAdd = Button(window, text = "ADD", command=RecognizeDigit)

buttonDelete.grid(row=3, column=0)
buttonAdd.grid(row=3, column=1)

# Add events to canva
canvas.bind('<Button-1>', locate_xy)
canvas.bind('<B1-Motion>', addLine)


window.mainloop()